In [ ]:
import specpy

from pipeline2 import AcquisitionPipeline
from pipeline2.imspector import get_current_stage_coords
from pipeline2.callback_buildingblocks.static_settings import JSONSettingsLoader
from pipeline2.callback_buildingblocks.parameter_filtering import DefaultLocationRemover
from pipeline2.callback_buildingblocks.coordinate_value_wrappers import StageOffsetsSettingsGenerator
from pipeline2.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from pipeline2.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
from pipeline2.taskgeneration import AcquisitionTaskGenerator
from pipeline2.callback_buildingblocks.stitched_data_selection import StitchedNewestDataSelector
from pipeline2.detection.roi_detection import ROIDetectorWrapper

%matplotlib inline

In [ ]:
# where to save & whether to save combined HDF5 file
save_folder = 'D:/AUTOMATION/TEST/20241126_stitching_test'
save_hdf5 = True

# path of measurement parameters (dumped to JSON file)
measurement_parameters = 'test_configs/20241010_overview_3d_640.json'

# yx FOV size
move_size = [45e-6, 45e-6]

In [ ]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

In [ ]:
# build pipeline object (1 level: 'field')
pipeline = AcquisitionPipeline('multipoint-acquisition', save_folder, ['field'], save_combined_hdf5=save_hdf5)

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('field',
    DefaultLocationRemover(JSONSettingsLoader(measurement_parameters, as_measurements=False)),
    StageOffsetsSettingsGenerator(SpiralOffsetGenerator(move_size, start_coords[1:]))
    )

dummy_callback = AcquisitionTaskGenerator('field',
    ROIDetectorWrapper(StitchedNewestDataSelector(pipeline, 'field', register_tiles=True), lambda x: [])
)
# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, 'field')

pipeline.add_callback(dummy_callback, 'field')

# set maximum number of acquisitions before stop
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(10))

# start with initial task from callback
pipeline.run(next_position_generator)